1. Veri okuma

Veri setinden gereksiz sütun ('Patient_ID') analiz için gereksiz olduğu için çıkarıldı.

In [81]:
import pandas as pd

file_path = "datasets/thyroid_cancer_risk_data.csv"
df = pd.read_csv(file_path)

# Gereksiz sütunları çıkarımı
df.drop(columns=["Patient_ID"], inplace=True)

print(df.info())

print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212691 entries, 0 to 212690
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Age                  212691 non-null  int64  
 1   Gender               212691 non-null  object 
 2   Country              212691 non-null  object 
 3   Ethnicity            212691 non-null  object 
 4   Family_History       212691 non-null  object 
 5   Radiation_Exposure   212691 non-null  object 
 6   Iodine_Deficiency    212691 non-null  object 
 7   Smoking              212691 non-null  object 
 8   Obesity              212691 non-null  object 
 9   Diabetes             212691 non-null  object 
 10  TSH_Level            212691 non-null  float64
 11  T3_Level             212691 non-null  float64
 12  T4_Level             212691 non-null  float64
 13  Nodule_Size          212691 non-null  float64
 14  Thyroid_Cancer_Risk  212691 non-null  object 
 15  Diagnosis        

2. Hedef Değişkeni (Thyroid_Cancer_Risk) Sayısala Çevirme

In [82]:
# Hedef değişkeni dönüştürme
risk_mapping = {"Low": 0, "Medium": 1, "High": 2}
df["Thyroid_Cancer_Risk"] = df["Thyroid_Cancer_Risk"].map(risk_mapping)

# Dönüşümü kontrol edelim
print(df["Thyroid_Cancer_Risk"].unique())

[0 1 2]


2.1. Kategorik değişkenleri sayısala dönüştürme

In [83]:
# Yes → 1, No → 0 olarak kodlama
binary_columns = ["Family_History", "Radiation_Exposure", "Iodine_Deficiency", "Smoking", "Obesity", "Diabetes"]
df[binary_columns] = df[binary_columns].applymap(lambda x: 1 if x == "Yes" else 0)

# Dönüşüm başarılı mı kontrol edelim
print(df[binary_columns].head())

   Family_History  Radiation_Exposure  Iodine_Deficiency  Smoking  Obesity  \
0               0                   1                  0        0        0   
1               0                   1                  0        0        0   
2               0                   0                  0        0        0   
3               0                   0                  0        0        0   
4               1                   1                  0        0        0   

   Diabetes  
0         0  
1         0  
2         0  
3         0  
4         0  


/var/folders/dm/hckhrzb5207cp2xy2c6ts38c0000gn/T/ipykernel_42699/4046470581.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[binary_columns] = df[binary_columns].applymap(lambda x: 1 if x == "Yes" else 0)


2.2. Çoklu Kategorik değişkenleri sayısala dönüştürme

In [84]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
for col in ["Gender", "Country", "Ethnicity"]:
    df[col] = encoder.fit_transform(df[col])

df = pd.get_dummies(df, columns=["Gender", "Country", "Ethnicity"], drop_first=True)

2.3. Sayısal değişkenleri ölçeklendirme

In [85]:
from sklearn.preprocessing import StandardScaler

# StandardScaler ile ölçeklendirme
scaler = StandardScaler()
numeric_features = ["TSH_Level", "T3_Level", "T4_Level", "Nodule_Size"]
df.loc[:, numeric_features] = scaler.fit_transform(df[numeric_features])

In [86]:
import numpy as np

unique_classes, class_counts = np.unique(df["Thyroid_Cancer_Risk"], return_counts=True)
print("Sınıf Dağılımı:")
for cls, count in zip(unique_classes, class_counts):
    print(f"Sınıf {cls}: {count} örnek")

Sınıf Dağılımı:
Sınıf 0: 108388 örnek
Sınıf 1: 72400 örnek
Sınıf 2: 31903 örnek


3. Eksik veri kontrolü

In [87]:
# Eksik veri kontrolü
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

Series([], dtype: int64)


3.1. Eksik verileri ortalama ile doldurma

In [88]:
# Mode değerini kontrol et ve ona göre doldur
if not df["Diagnosis"].mode().empty:
    df["Diagnosis"] = df["Diagnosis"].fillna(df["Diagnosis"].mode()[0])
else:
    df["Diagnosis"] = df["Diagnosis"].fillna("Benign")  # Varsayılan değer olarak 'Benign' atanabilir.

In [89]:
df.drop(columns=["Diagnosis"], inplace=True)

3. Yeni verisetini oluşturma ve kaydetme

Ön işlenmiş veriyi tekrar yeni bir veri setine çevrildi ve analiz - modelleme için kaydedildi.
Ön işlenmiş veri kümesi yeni bir CSV dosyası olarak kaydedildi.

In [90]:
processed_file_path = "datasets/processed_thyroid_cancer_data.csv"
df.to_csv(processed_file_path, index=False)

print(f"Veri seti başarıyla kaydedildi: {processed_file_path}")

Veri seti başarıyla kaydedildi: datasets/processed_thyroid_cancer_data.csv


4. Veri setini küçük parçaya ayırma (%10)

In [91]:
import pandas as pd

# Ön işleme yapılmış veri setini yükleyelim
file_path = "datasets/new_processed_thyroid_cancer_data.csv"  # Orijinal dosya adı
df = pd.read_csv(file_path)

# Veri setinin %10'unu seçerek daha küçük bir veri seti oluşturalım
df_sampled = df.sample(frac=0.1, random_state=42)

# Yeni dosya adı belirleyelim
small_dataset_path = "datasets/small_processed_thyroid_cancer_data.csv"

# Küçültülmüş veri setini CSV formatında kaydedelim
df_sampled.to_csv(small_dataset_path, index=False)

print(f"Küçültülmüş veri seti kaydedildi: {small_dataset_path}")

Küçültülmüş veri seti kaydedildi: datasets/small_processed_thyroid_cancer_data.csv
